In [5]:
import pandas as pd

crypto_df = pd.read_csv('crypto-markets.csv')
crypto_df.head()

,slug,asset,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
0,target-coin,TGT,Target Coin,29-09-2017,607,0.028961,0.054766,0.028961,0.041777,69996,0.0,0.4966,0.03
1,target-coin,TGT,Target Coin,30-09-2017,607,0.041783,0.046196,0.031435,0.031744,5725,0.0,0.0209,0.01
2,target-coin,TGT,Target Coin,01-10-2017,607,0.031761,0.035957,0.021040,0.028385,5012,0.0,0.4924,0.01
3,target-coin,TGT,Target Coin,02-10-2017,607,0.028375,0.054595,0.020417,0.022525,8010,0.0,0.0617,0.03
4,target-coin,TGT,Target Coin,03-10-2017,607,0.022527,0.032225,0.020211,0.020359,1787,0.0,0.0123,0.01


In [9]:
import numpy as np
import pandas as pd

assetsCode = ['BTC','ETH','XRP','LTC']
# qui convertiamo il prezzo di apertura, chiusura, quello più alto e quello più basso in valuta GBP da quella del dollaro 

# prendiamo in considerazione solo le cripto-valute presenti nella list ['BTC','ETH','XRP','LTC']
crypto_df['open'] = crypto_df[['open', 'asset']].apply(lambda x: (float(x[0]) * 0.80) if x[1] in assetsCode else np.nan, axis=1)
crypto_df['close'] = crypto_df[['close', 'asset']].apply(lambda x: (float(x[0]) * 0.80) if x[1] in assetsCode else np.nan, axis=1)
crypto_df['high'] = crypto_df[['high', 'asset']].apply(lambda x: (float(x[0]) * 0.80) if x[1] in assetsCode else np.nan, axis=1)
crypto_df['low'] = crypto_df[['low', 'asset']].apply(lambda x: (float(x[0]) * 0.80) if x[1] in assetsCode else np.nan, axis=1)

# eliminiamo le righe con i valori nulli
crypto_df.dropna(inplace=True)

# resettiamo gli indici del dataset
crypto_df.reset_index(drop=True ,inplace=True)
crypto_df.head()

,slug,asset,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
0,bitcoin,BTC,Bitcoin,28-04-2013,1,86.5920,87.0272,84.5440,85.8944,0,1.500520e+09,0.5438,3.88
1,bitcoin,BTC,Bitcoin,29-04-2013,1,86.0416,94.3936,85.7600,92.5056,0,1.491160e+09,0.7813,13.49
2,bitcoin,BTC,Bitcoin,30-04-2013,1,92.1600,94.0352,85.7920,88.9600,0,1.597780e+09,0.3843,12.88
3,bitcoin,BTC,Bitcoin,01-05-2013,1,88.9600,89.5296,68.9408,74.8736,0,1.542820e+09,0.2882,32.17
4,bitcoin,BTC,Bitcoin,02-05-2013,1,74.4832,80.3840,59.0592,67.3344,0,1.292190e+09,0.3881,33.32


In [10]:
crypto_df.drop(labels=['slug', 'ranknow', 'volume', 'market', 'close_ratio', 'spread'], inplace=True, axis=1)
crypto_df.head()

,asset,name,date,open,high,low,close
0,BTC,Bitcoin,28-04-2013,86.5920,87.0272,84.5440,85.8944
1,BTC,Bitcoin,29-04-2013,86.0416,94.3936,85.7600,92.5056
2,BTC,Bitcoin,30-04-2013,92.1600,94.0352,85.7920,88.9600
3,BTC,Bitcoin,01-05-2013,88.9600,89.5296,68.9408,74.8736
4,BTC,Bitcoin,02-05-2013,74.4832,80.3840,59.0592,67.3344


In [11]:
import sqlite3

# connessione al SQLite database file, 
conn = sqlite3.connect('session.db')
print(conn)


# Se già esiste una tabella con il nome Crypto procediamo a cancellarla con drop 
try:
    conn.execute('DROP TABLE IF EXISTS `Crypto` ')
except Exception as e:
    raise(e)
finally:
    print('Table dropped')

    
# Creiamo la nuova tablella Crypto
try:
    conn.execute('''
         CREATE TABLE Crypto
         (ID         INTEGER PRIMARY KEY,
         ASSET       TEXT NOT NULL,
         NAME        TEXT    NOT NULL,
         Date        datetime,
         Open        Float DEFAULT 0,
         High        Float DEFAULT 0,
         Low         Float DEFAULT 0,
         Close       Float DEFAULT 0);''')
    print ("Table created successfully");
except Exception as e:
    print(str(e))
    print('Table Creation Failed!!!!!')
finally:
    conn.close() # chiudiamo la database connection

Table dropped
Table created successfully


In [12]:
# conversione del pandas dtaframe to list of list
crypto_list = crypto_df.values.tolist()

# creiamo una connessione per l'i Insert dei crypto data in SQL DB
conn = sqlite3.connect('session.db')

# creiamo un cursore - ci aiuterà nel querying SQL DB
cur = conn.cursor()

try:
    
    #useremo il simbolo ? per rappresentare ogni colonna del VALUES.
    cur.executemany("INSERT INTO Crypto(ASSET, NAME, Date, Open, High, Low, Close) VALUES (?,?,?,?,?,?,?)", crypto_list)
    conn.commit()
    print('Data Inserted Successfully')
except Exception as e:
    print(str(e))
    print('Data Insertion Failed')
finally:
    
    conn.close()



Data Inserted Successfully


In [13]:
print (crypto_list)


[['BTC', 'Bitcoin', '28-04-2013', 108.24000000000001, 108.78399999999999, 105.68, 107.36800000000001], ['BTC', 'Bitcoin', '29-04-2013', 107.552, 117.99200000000002, 107.2, 115.632], ['BTC', 'Bitcoin', '30-04-2013', 115.2, 117.54400000000001, 107.24000000000001, 111.2], ['BTC', 'Bitcoin', '01-05-2013', 111.2, 111.91199999999999, 86.176, 93.592], ['BTC', 'Bitcoin', '02-05-2013', 93.104, 100.48, 73.824, 84.168], ['BTC', 'Bitcoin', '03-05-2013', 85.0, 86.504, 63.28, 78.2], ['BTC', 'Bitcoin', '04-05-2013', 78.48, 92.0, 74.0, 90.0], ['BTC', 'Bitcoin', '05-05-2013', 90.32000000000001, 95.04, 85.712, 92.72800000000001], ['BTC', 'Bitcoin', '06-05-2013', 92.784, 99.72800000000001, 85.31200000000001, 89.84], ['BTC', 'Bitcoin', '07-05-2013', 89.80000000000001, 90.75200000000001, 78.16000000000001, 89.2], ['BTC', 'Bitcoin', '08-05-2013', 87.68, 92.62400000000001, 87.68, 90.856], ['BTC', 'Bitcoin', '09-05-2013', 90.56, 90.768, 87.40800000000002, 90.13600000000001], ['BTC', 'Bitcoin', '10-05-2013', 9